# 实验二 拉伸法测杨氏弹性模量

Copyright © 2024 Zjl37

In [ ]:
# 请安装 uncertainties 包：
# pip install uncertainties

import uncertainties
from uncertainties import ufloat, unumpy
import statistics as stat
import csv
import math

In [ ]:
data = [[x.strip() for x in ln] for ln in csv.reader(open('./data-raw.csv'))]
L_, H_, D_ = map(float, data[2][:3]) # (mm)
dm_ = 1.00 # (kg)
d0 = float(data[5][1]) # (mm) 螺旋测微器的零差 注：当可动刻度的零刻度线在固定刻度的零刻度线上方时为正
d视 = [float(di) for di in data[7][1:7]] # (mm)
xup = [float(x) for x in data[12][1:] if x != ''] # (mm)
xdown = [float(x) for x in data[13][1:] if x != ''] # (mm)


In [ ]:
if not any(xdown):
    raise ValueError('请先在 ./data-raw.csv 文件中填写你的实验数据。')

### 一些测量工具的参考参数

| 量具名称 | 测量的物理量 | 误差限（$\Delta_仪$） |
| :-: | :-: | :-: |
| 钢卷尺 | $L, H$ | 0.8 mm |
| 游标卡尺 | $D$ | 0.02 mm |
| 螺旋测微器 | $d$ | 0.004 mm |
| 数字拉力计 | $m$ | 0.005 kg |
| 标尺 | $\Delta x$ | 0.5 mm |


In [ ]:
C = math.sqrt(3)

UL = UH = 0.8/C
UD = 0.02/C
Udm = 0.005/C

L = ufloat(L_, UL) # (mm)
H = ufloat(H_, UH) # (mm)
D = ufloat(D_, UD) # (mm)
dm = ufloat(dm_, Udm) # (kg)

In [ ]:
UL, UD, Udm

In [ ]:
print(f"""
L = {L:P} mm
H = {H:P} mm
D = {D:P} mm
Δm = {dm:P} kg
""")

In [ ]:
d_ = d0 + stat.mean(d视) # (mm)

nd = len(d视)
Sd视 = stat.stdev(d视)/math.sqrt(nd)
ud = 0.004/C
Ud = math.sqrt(Sd视**2 + ud**2)
d_, Sd视, ud, Ud

In [ ]:
d = ufloat(d_, Ud)
print(f"d = {d:P} mm")

In [ ]:
xup = unumpy.uarray(xup, 0.5/C) # (mm)
xdown = unumpy.uarray(xdown, 0.5/C) # (mm)
# 计算 x_i
x = (xup + xdown)/2 # (mm)
# 逐差法，计算 Δx_i
n = len(x)//2
dx = x[-n:] - x[:n]

In [ ]:
# 将计算得的 x_i 和 Δx_i 写到文件

def my_round_to_even(x, digits=1):
    import decimal
    return round(round(decimal.Decimal(x), 14), digits)

data[7].insert(7, str(my_round_to_even(stat.mean(d视), 3)))
data[14][1:] = [str(my_round_to_even(x.nominal_value)) for x in x]
data[15][1:] = [str(my_round_to_even(x.nominal_value)) for x in dx]

with open('./data.csv', 'w') as f:
    csv.writer(f).writerows(data)

In [ ]:
dx = dx.mean()/n # (mm)
dx_, Udx = dx.nominal_value, dx.std_dev
print(f"Δx = {dx:P}")

In [ ]:
g = 9.7887 # (m/s²)

In [ ]:
# 计算杨氏模量 E

E = 8*dm*g*(0.001*L)*(0.001*H)/(math.pi*(0.001*d)**2*(0.001*D)*(0.001*dx)) # (N/m²)
print(f"E = {E:P}")

In [ ]:
# 相对不确定度
EE = E.std_dev / E.nominal_value
EE

In [ ]:
# 以上程序使用 uncertainties 库自动计算合成不确定度
# 可以验证，其结果与使用公式算出的相符：

Ej = [UL/L_, UH/H_, UD/D_, Udm/dm_, 2*Ud/d_, Udx/dx_]

math.sqrt(sum(e**2 for e in Ej))

In [ ]:
print('合成相对不确定度的各分量')
sorted(zip(['L', 'H', 'D', 'Δm', 'd', 'Δx'], Ej), key=lambda p: -p[1])

In [ ]:
print('合成不确定度的各分量')
sorted(E.error_components().items(), key=lambda p: -p[1])